In [1]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

import glob
import numpy as np
import os

In [2]:
# Upload and play movie
F147 = cm.load('C:/Users/pugavin/BCB-330-2023-05/data/2p_raw/F147/F147_20210526_fish4_blk1_LT_9dpf_00001.tif')
print(F147.shape)
F147.play()

<tifffile.TiffFile 'F147_20210526_fi…_9dpf_00001.tif'> asarray failed to reshape (41990, 320, 256) to (26000, 2, 320, 256)


(41990, 320, 256)


In [3]:
# Save and reload movie 
# See GitHub Issue #377 - https://github.com/flatironinstitute/CaImAn/issues/377#issuecomment-426740429
F147.save('F147.tif')
F147 = cm.load('F147.tif')

In [4]:
# Set parameters
opts_params = {
    'use_cuda': True # use GPU
}
opts = params.CNMFParams(params_dict=opts_params)

In [5]:
# Start cluster
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local',
    n_processes=None,
    single_thread=False
)

In [6]:
# Motion correction
mc = MotionCorrect(
    ['F147.tif'],
    dview=dview,
    **opts.get_group('motion')
)
mc.motion_correct(save_movie=True)

In [7]:
# Compare the original and motion corrected movies
F147_mc = cm.load(mc.mmap_file)
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0
ds_ratio = 1
cm.concatenate([F147.resize(1, 1, ds_ratio) - mc.min_mov * mc.nonneg_movie,
                F147_mc.resize(1, 1, ds_ratio)], axis=2).play()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.72s/it]


In [8]:
# Save and load memory mapping
mmap = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C')
Yr, dims, T = cm.load_memmap(mmap)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

In [9]:
# Restart cluster
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local',
    n_processes=None,
    single_thread=False
)

In [10]:
# CNMF
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)
Cn = cm.local_correlations(images.transpose(1, 2, 0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

In [11]:
# Component evaluation
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
cnm.estimates.plot_contours_nb(img=Cn, idx=cnm.estimates.idx_components)
cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components)
if len(cnm.estimates.idx_components_bad) > 0:
    cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components_bad)

GPU run not requested, disabling use of GPUs
USING MODEL (keras API): C:\Users\pugavin\caiman_data\model\cnn_model.json
1/1 [==============================] - 0s 156ms/step


In [12]:
# Prepare and view final results
cnm.estimates.detrend_df_f()
cnm.estimates.select_components(use_object=True)
cnm.estimates.nb_view_components(img=Cn, denoised_color='red')
cnm.estimates.play_movie(images)

movie([[[ 88.3392   ,  89.06931  ,  86.0717   , ...,   9.44072  ,
           9.488541 ,   9.476692 ],
        [ 84.29774  ,  83.80311  ,  86.584755 , ...,   6.2607117,
           6.348877 ,   6.3632965],
        [ 87.96576  ,  85.30035  ,  86.23332  , ...,   4.82193  ,
           4.827194 ,   4.8008804],
        ...,
        [ 86.440796 ,  84.88653  ,  92.32727  , ...,   6.5406647,
           6.5891266,   6.582199 ],
        [ 88.71199  ,  82.79227  ,  87.32204  , ...,   8.150719 ,
           8.127296 ,   8.090027 ],
        [ 88.71199  ,  82.79227  ,  87.32204  , ...,   8.140747 ,
           8.128571 ,   8.103081 ]],

       [[ 72.70942  ,  70.170555 ,  71.560974 , ...,  -9.541008 ,
          -9.493172 ,  -9.50502  ],
        [ 72.70942  ,  70.170555 ,  71.560974 , ...,  -9.568314 ,
          -9.480125 ,  -9.465698 ],
        [ 72.73377  ,  71.3659   ,  70.62903  , ...,  -8.190102 ,
          -8.184837 ,  -8.211159 ],
        ...,
        [ 71.0361   ,  70.585594 ,  70.91259  , ...,  

In [13]:
# Save hdf5 file
cnm.save('F147_results.hdf5')

In [14]:
# Stop cluster and clean up logs
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)